In [1]:
# Import the necessary modules for the protocol
import ee as ee
ee.Initialize()
import pandas as pd
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
import numpy as np
from itertools import combinations

In [22]:
# Import the data and view a summary of it
importedData = pd.read_csv('20201012_Wood_Density_Project_Merged_sampled_dataset_Diversity_Pixel.csv');
# define a vector of the variables we are targeting
selectedVariables = ["Aridity_Index","CHELSA_Annual_Mean_Temperature","CHELSA_Annual_Precipitation","CHELSA_Isothermality","CHELSA_Max_Temperature_of_Warmest_Month","CHELSA_Mean_Diurnal_Range","CHELSA_Mean_Temperature_of_Coldest_Quarter","CHELSA_Mean_Temperature_of_Driest_Quarter","CHELSA_Mean_Temperature_of_Warmest_Quarter","CHELSA_Mean_Temperature_of_Wettest_Quarter","CHELSA_Min_Temperature_of_Coldest_Month","CHELSA_Precipitation_Seasonality","CHELSA_Precipitation_of_Coldest_Quarter","CHELSA_Precipitation_of_Driest_Month","CHELSA_Precipitation_of_Driest_Quarter","CHELSA_Precipitation_of_Warmest_Quarter","CHELSA_Precipitation_of_Wettest_Month","CHELSA_Precipitation_of_Wettest_Quarter","CHELSA_Temperature_Annual_Range","CHELSA_Temperature_Seasonality","Depth_to_Water_Table","EarthEnvCloudCover_MODCF_interannualSD","EarthEnvCloudCover_MODCF_intraannualSD","EarthEnvCloudCover_MODCF_meanannual","EarthEnvTopoMed_Eastness","EarthEnvTopoMed_Elevation","EarthEnvTopoMed_Northness","EarthEnvTopoMed_ProfileCurvature","EarthEnvTopoMed_Roughness","EarthEnvTopoMed_Slope","EarthEnvTopoMed_TopoPositionIndex","SG_Absolute_depth_to_bedrock","WorldClim2_SolarRadiation_AnnualMean","WorldClim2_WindSpeed_AnnualMean","WorldClim2_H2OVaporPressure_AnnualMean","NDVI","EVI","Lai","Fpar","Npp","Tree_Density","PET","SG_Clay_Content_0_100cm","SG_Coarse_fragments_0_100cm","SG_Sand_Content_0_100cm","SG_Silt_Content_0_100cm","SG_Soil_pH_H2O_0_100cm","LandCoverClass_Cultivated_and_Managed_Vegetation","LandCoverClass_Urban_Builtup","Human_Disturbance","treecover2000","Nitrogen","CanopyHeight","cropland","grazing","pasture","rangeland","Fire_Frequency","cnRatio","Cation"]

importedData = importedData[selectedVariables]
importedData.info()
importedData.describe()

# Instantiate the composite that was used to sample the points
fullCompositeImage = ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
presentCompositeImage = fullCompositeImage.select(selectedVariables)
print('Composite Bands',presentCompositeImage.bandNames().getInfo())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549794 entries, 0 to 549793
Data columns (total 60 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Aridity_Index                                     549002 non-null  float64
 1   CHELSA_Annual_Mean_Temperature                    549518 non-null  float64
 2   CHELSA_Annual_Precipitation                       549518 non-null  float64
 3   CHELSA_Isothermality                              549518 non-null  float64
 4   CHELSA_Max_Temperature_of_Warmest_Month           549518 non-null  float64
 5   CHELSA_Mean_Diurnal_Range                         549518 non-null  float64
 6   CHELSA_Mean_Temperature_of_Coldest_Quarter        549518 non-null  float64
 7   CHELSA_Mean_Temperature_of_Driest_Quarter         549518 non-null  float64
 8   CHELSA_Mean_Temperature_of_Warmest_Quarter        549518 non-null  float64
 9   CHEL

In [23]:
# Input the proportion of variance that you would like to cover when running the script
propOfVariance = 90

In [36]:
def assessExtrapolation(importedData, compositeImage, propOfVariance):
    
    # Excise the columns of interest from the data frame
    variablesOfInterest = importedData.dropna().drop([], axis=1)
    
    # Compute the mean and standard deviation of each band, then standardize the point data
    meanVector = variablesOfInterest.mean()
    stdVector = variablesOfInterest.std()
    standardizedData = (variablesOfInterest-meanVector)/stdVector
    
    # Then standardize the composite from which the points were sampled
    meanList = meanVector.tolist()
    stdList = stdVector.tolist()
    bandNames = list(meanVector.index)
    meanImage = ee.Image(meanList).rename(bandNames)
    stdImage = ee.Image(stdList).rename(bandNames)
    standardizedImage = compositeImage.subtract(meanImage).divide(stdImage)
    
    # Run a PCA on the point samples
    pcaOutput = PCA()
    pcaOutput.fit(standardizedData)
    
    # Save the cumulative variance represented by each PC
    cumulativeVariance = np.cumsum(np.round(pcaOutput.explained_variance_ratio_, decimals=4)*100)
    
    # Make a list of PC names for future organizational purposes
    pcNames = ['PC'+str(x) for x in range(1,variablesOfInterest.shape[1]+1)]
    
    # Get the PC loadings as a data frame
    loadingsDF = pd.DataFrame(pcaOutput.components_,columns=[str(x)+'_Loads' for x in bandNames],index=pcNames)
    
    # Get the original data transformed into PC space
    transformedData = pd.DataFrame(pcaOutput.fit_transform(standardizedData,standardizedData),columns=pcNames)
    
    # Make principal components images, multiplying the standardized image by each of the eigenvectors
    # Collect each one of the images in a single image collection;
    
    # First step: make an image collection wherein each image is a PC loadings image
    listOfLoadings = ee.List(loadingsDF.values.tolist());
    eePCNames = ee.List(pcNames)
    zippedList = eePCNames.zip(listOfLoadings)
    def makeLoadingsImage(zippedValue):
        return ee.Image.constant(ee.List(zippedValue).get(1)).rename(bandNames).set('PC',ee.List(zippedValue).get(0))
    loadingsImageCollection = ee.ImageCollection(zippedList.map(makeLoadingsImage))
    
    # Second step: multiply each of the loadings image by the standardized image and reduce it using a "sum"
    # to finalize the matrix multiplication
    def finalizePCImages(loadingsImage):
        return ee.Image(loadingsImage).multiply(standardizedImage).reduce('sum').rename([ee.String(ee.Image(loadingsImage).get('PC'))]).set('PC',ee.String(ee.Image(loadingsImage).get('PC')))
    principalComponentsImages = loadingsImageCollection.map(finalizePCImages)
    
    # Choose how many principal components are of interest in this analysis based on amount of
    # variance explained
    numberOfComponents = sum(i < propOfVariance for i in cumulativeVariance)+1
    print('Number of Principal Components being used:',numberOfComponents)
    
    # Compute the combinations of the principal components being used to compute the 2-D convex hulls
    tupleCombinations = list(combinations(list(pcNames[0:numberOfComponents]),2))
    print('Number of Combinations being used:',len(tupleCombinations))
    
    # Generate convex hulls for an example of the principal components of interest
    cHullCoordsList = list()
    for c in tupleCombinations:
        firstPC = c[0]
        secondPC = c[1]
        outputCHull = ConvexHull(transformedData[[firstPC,secondPC]])
        listOfCoordinates = transformedData.loc[outputCHull.vertices][[firstPC,secondPC]].values.tolist()
        flattenedList = [val for sublist in listOfCoordinates for val in sublist]
        cHullCoordsList.append(flattenedList)
    
    # Reformat the image collection to an image with band names that can be selected programmatically
    pcImage = principalComponentsImages.toBands().rename(pcNames)
    
    # Generate an image collection with each PC selected with it's matching PC
    listOfPCs = ee.List(tupleCombinations)
    listOfCHullCoords = ee.List(cHullCoordsList)
    zippedListPCsAndCHulls = listOfPCs.zip(listOfCHullCoords)
    
    def makeToClassifyImages(zippedListPCsAndCHulls):
        imageToClassify = pcImage.select(ee.List(zippedListPCsAndCHulls).get(0)).set('CHullCoords',ee.List(zippedListPCsAndCHulls).get(1))
        classifiedImage = imageToClassify.rename('u','v').classify(ee.Classifier.spectralRegion([imageToClassify.get('CHullCoords')]))
        return classifiedImage
    classifedImages = ee.ImageCollection(zippedListPCsAndCHulls.map(makeToClassifyImages))
    finalImageToExport = classifedImages.sum().divide(ee.Image.constant(len(tupleCombinations)))
    
    return finalImageToExport



In [37]:
# Apply the function
finalImageToExport = assessExtrapolation(importedData, presentCompositeImage, propOfVariance)

Number of Principal Components being used: 21
Number of Combinations being used: 210


In [38]:
print(finalImageToExport.getInfo())

{'type': 'Image', 'bands': [{'id': 'classification', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -43920820900200448, 'max': 43920820900200448}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [39]:
# Define the export boundary
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False);

In [40]:
# Export the image to test it
myTask = ee.batch.Export.image.toAsset(
    image = finalImageToExport,
    description = 'Wood_Density_Representativeness_IntExt_Export',
    assetId = 'users/leonidmoore/WoodDensityProject/IntExt_Map/Wood_Density_Representativeness_IntExt_Result',
    region = unboundedGeo,
    maxPixels = 1e13,
    crs = 'EPSG:4326',
    crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90]
)
# start the task and show the status on Google earth engine UI
myTask.start()
myTask.status()

{'state': 'READY',
 'description': 'Wood_Density_Representativeness_IntExt_Export',
 'creation_timestamp_ms': 1607434137149,
 'update_timestamp_ms': 1607434137149,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'A6MD3Q5HEJUVHG3XUEIEPREB',
 'name': 'projects/earthengine-legacy/operations/A6MD3Q5HEJUVHG3XUEIEPREB'}

In [42]:
# export to google drive
driveTask = ee.batch.Export.image.toDrive(
    image = finalImageToExport,
    description = 'Wood_Density_IntExt_Export_to_Drive',
    fileNamePrefix = 'Wood_Density_Representativeness_IntExt_Map',
    region = unboundedGeo,
    crs = 'EPSG:4326',
    crsTransform= [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
    maxPixels = 1e13
)
driveTask.start()
driveTask.status()

{'state': 'READY',
 'description': 'Wood_Density_IntExt_Export_to_Drive',
 'creation_timestamp_ms': 1607435793041,
 'update_timestamp_ms': 1607435793041,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '3FBWWJCOXSAMQQVQCXRMT6IK',
 'name': 'projects/earthengine-legacy/operations/3FBWWJCOXSAMQQVQCXRMT6IK'}